#### <font color=red>目前,模型和规则中,最后一批人的训练速度非常慢,且容易出现OOM情况<br>1.取消所有operation操作;<br>2.多使用persist和broadcast;

In [32]:
import os,time,gc,re
import pyspark
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext,SparkConf,SQLContext
from pyspark.sql import Window
from pyspark.conf import SparkConf
from pyspark.sql.functions import mean,stddev,log
import pyspark.sql.functions as fn
warnings.filterwarnings('ignore')

In [33]:
ft=time.time()

In [34]:
spark = pyspark.sql.SparkSession.builder.appName('LQ_SKD_D333').config('spark.shuffle.consolidateFiles','true').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

today=datetime.strftime(datetime.now(),'%Y-%m-%d')
print(today)

revise_types=['claim','routine_maintenance','accessory','accident_repair','general_repair',
       'first_maintance','service_act']

2020-08-08


In [35]:
def get_miss_cols(dff):
    tmp=dff.agg(*[(1-(fn.count(c)/fn.count('*'))).alias(c) for c in dff.columns])
    cols=tmp.columns
    leng=len(cols)
    lst=tmp.collect()
    miss_cols=[]
    for i in range(leng):
        v=[x[i] for x in lst][0]
        if v>0:
            miss_cols.append(cols[i])
    return miss_cols

def check_df(df):
    print(df.count())
    print(df.select('vin','repair_date').dropDuplicates().count())
    print(df.select('vin').distinct().count())
    
def check_ziduan(df1):
    cols=df1.columns
    cols.remove('vin')
    for x in cols:
        print(x)
        print([y[0] for y in df1.select(x).distinct().collect()][:10])
        print('-'*30)

def get_user(df1):
    df1=df1.withColumn('car_body_type',substring('vin',4,1)).\
    withColumn('gearbox',substring('vin',5,1)).withColumn('crew_protection_system',substring('vin',6,1)).\
    withColumn('car_class',substring('vin',7,2)).withColumn('output_year',substring('vin',10,1)).\
    withColumn('assembly_factory',substring('vin',11,1))

    df1=df1.withColumn('user_tag',concat_ws(',',col('car_body_type'),col('gearbox'),col('crew_protection_system'),col('car_class'),col('output_year'),col('assembly_factory')))
    return df1

def check_time(s1):
    print('run time is:',(time.time()-s1)/60)

In [36]:
def fenlei(dff,today):
    
    dff=dff.withColumn('purchase_date',to_date(col('purchase_date'))).\
    withColumn('repair_date',to_date(col('repair_date'))).\
    withColumn('today',to_date(lit(today))).\
    withColumn('car_age',datediff(col('today'),col('purchase_date'))).\
    withColumn('car_age_bin',ceil(col('car_age')/180))
    
    if 'changgui_times' in dff.columns:
        dff=dff.drop('changgui_times')
        
    tmp=dff.filter(col('first_maintance')!=1)
    tmp=tmp.filter(col('routine_maintenance')==1).select('vin','repair_date').dropDuplicates().groupBy('vin').agg(count('repair_date').alias('changgui_times'))
    dff=dff.join(tmp,on='vin',how='left')
    dff=dff.withColumn('changgui_times',when(col('changgui_times').isNull(),lit(0)).otherwise(col('changgui_times')))
    
    if 'shoubao_times' in dff.columns:
        dff=dff.drop('shoubao_times')
    
    tmp=dff.filter(col('first_maintance')==1).select('vin').distinct().withColumn('shoubao_times',lit(1))
    dff=dff.join(tmp,on='vin',how='left')
    dff=dff.withColumn('shoubao_times',when(col('shoubao_times').isNull(),lit(0)).otherwise(col('shoubao_times')))
    
    dff=dff.withColumn('the_group',when((col('shoubao_times')==1)&(col('changgui_times')==0),lit('有首保,无常规,预测第1次常规')).\
                       when((col('shoubao_times')==1)&(col('changgui_times')==1),lit('有首保,1常规,预测第2次常规')).\
                       when((col('shoubao_times')==1)&(col('changgui_times')==2),lit('有首保,2次常规,预测第3次常规')).\
                       when((col('shoubao_times')==1)&(col('changgui_times')>=3),lit('3次及以上常规,预测接下去常规')).\
                       when((col('shoubao_times')==0)&(col('changgui_times')==1),lit('无首保,1次常规,预测第2次常规(购车)')).\
                       when((col('shoubao_times')==0)&(col('changgui_times')==2),lit('无首保,2次常规,预测第3次常规')).\
                       when((col('shoubao_times')==0)&(col('changgui_times')>=3),lit('3次及以上常规,预测接下去常规')).\
                       when((col('shoubao_times')==0)&(col('changgui_times')==0)&(col('car_age_bin')<=3),lit('无保养记录,预测首保')).\
                       when((col('shoubao_times')==0)&(col('changgui_times')==0)&(col('car_age_bin')>3),lit('无保养习惯')).\
                       otherwise(lit('错误提示:没有被分配到人群!')))

    print('查看是否有没有被分配到的人群')
    c=dff.select('vin','the_group').dropDuplicates()
    print(c.count())
    print(c.select('vin').distinct().count())
    mis=get_miss_cols(c)
    print(mis)
    
    print('查看对应的每一类人群的VIN数量')
    tmp=dff.select('vin','the_group').dropDuplicates().groupBy('the_group').agg(count('vin').alias('cnt'))
    print(tmp.show())
    
    baoyangdf1=dff.filter((col('first_maintance')==1)|(col('routine_maintenance')==1))
    baoyang_vin=baoyangdf1.select('vin').distinct().dropDuplicates().withColumn('have_baoyang',lit(1))
    
    nobaoyang1=dff.join(baoyang_vin,on='vin',how='left').withColumn('have_baoyang',when(col('have_baoyang').isNull(),lit(0)).otherwise(col('have_baoyang')))
    
    return baoyangdf1,nobaoyang1

In [37]:
def get_last_run(df):
    df=df.withColumn('purchase_date',to_date(col('purchase_date'))).withColumn('repair_date',to_date(col('repair_date')))
    df=df.withColumn('last_repair_date',lag(col('repair_date'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    df=df.withColumn('last_mile',lag(col('mile'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    df=df.withColumn('last_repair_date',when(col('last_repair_date').isNull(),col('purchase_date')).otherwise(col('last_repair_date'))).\
    withColumn('last_mile',when(col('last_mile').isNull(),lit(0)).otherwise(col('last_mile'))).\
    withColumn('daydiff',datediff(col('repair_date'),col('last_repair_date'))).\
    withColumn('milediff',col('mile')-col('last_mile')).\
    withColumn('dayofmile',col('milediff')/col('daydiff'))
    return df

In [38]:
def clear_rule(rule):
    
#     print('历史保养次数')
    if 'baoyang_times' in rule.columns:
        rule=rule.drop('baoyang_times')
    tmp=rule.select('vin','repair_date').dropDuplicates().groupBy('vin').agg(count('repair_date').alias('baoyang_times'))
    rule=rule.join(tmp,on=['vin'],how='left')
    
#     print('得到对应的上一次修理日期和公里数')
    rule=rule.withColumn('last_repair_date',lag(col('repair_date'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule=rule.withColumn('last_mile',lag(col('mile'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule=rule.withColumn('last_repair_date',when(col('last_repair_date').isNull(),col('purchase_date')).otherwise(col('last_repair_date')))
    rule=rule.withColumn('last_mile',when(col('last_mile').isNull(),lit(0)).otherwise(col('last_mile')))
    rule=rule.withColumn('daydiff',datediff(col('repair_date'),col('last_repair_date')))
    rule=rule.withColumn('milediff',col('mile')-col('last_mile'))
    rule=rule.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    rule=broadcast(rule)
    
#     print('开始天数差trick清洗')
    rule1=rule.filter(col('daydiff')>7.0)
    rule2=rule1.drop('baoyang_times')
    
    tmp=rule2.select('vin','repair_date').dropDuplicates().groupBy('vin').agg(count('repair_date').alias('baoyang_times'))
    rule2=rule2.join(tmp,on=['vin'],how='left')
    
    df11=rule2.filter(col('baoyang_times')==1)
    df11=df11.filter(col('daydiff')>7)
    df22=rule2.filter(col('baoyang_times')==2)
    tmp=df22.select('vin','repair_date','daydiff').dropDuplicates().groupBy('vin').agg(mean('daydiff')).withColumnRenamed('avg(daydiff)','daydiff_habit')
    df22=df22.join(tmp,on=['vin'],how='left').withColumn('right_habit',col('daydiff_habit')*0.5)
    df22=df22.filter(col('daydiff')>=col('right_habit'))
    df33=rule2.filter(col('baoyang_times')>=3)
    df33=df33.filter(col('daydiff')>14)

    df11=df11.drop('baoyang_times')
    df22=df22.drop('right_habit','daydiff_habit','baoyang_times')
    df33=df33.drop('baoyang_times')
    
    rule2=df11.unionAll(df22).unionAll(df33)
    if 'in_model' in rule2.columns:
        rule2=rule2.drop('in_model')
    rule2=rule2.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    rule2=broadcast(rule2)
    
    del df11
    del df22
    del df33
    del rule1
    del tmp
    
    rule=rule.join(rule2.select('vin').distinct().withColumn('cleared_vin',lit(0)),on='vin',how='left').\
    withColumn('cleared_vin',when(col('cleared_vin').isNull(),lit(1)).otherwise(col('cleared_vin')))
    
    cleared=rule.filter(col('cleared_vin')==1)
    if 'in_model' in cleared.columns:
        cleared=cleared.drop('in_model')
    if 'cleared_vin' in cleared.columns:
        cleared=cleared.drop('cleared_vin')
    cleared=cleared.withColumn('first_maintance',lit(0)).withColumn('routine_maintenance',lit(0))
    cleared=cleared.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    cleared=broadcast(cleared)
    
    del rule

    return rule2,cleared

In [39]:
def clear_rule_further(rule2):
    if 'baoyang_times' in rule2.columns:
        rule2=rule2.drop('baoyang_times')
    if 'last_repair_date' in rule2.columns:
        rule2=rule2.drop('last_repair_date')
    if 'last_mile' in rule2.columns:
        rule2=rule2.drop('last_mile')
    if 'daydiff' in rule2.columns:
        rule2=rule2.drop('daydiff')
    if 'milediff' in rule2.columns:
        rule2=rule2.drop('milediff')
    if 'dayofmile' in rule2.columns:
        rule2=rule2.drop('dayofmile')
    if 'baoyang_id' in rule2.columns:
        rule2=rule2.drop('baoyang_id')
    
#     print('清洗完成,重新跑last_run')
    tmp=rule2.select('vin','repair_date').dropDuplicates().groupBy('vin').agg(count('repair_date').alias('baoyang_times'))
    rule2=rule2.join(tmp,on=['vin'],how='left')

    rule2=rule2.withColumn('last_repair_date',lag(col('repair_date'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule2=rule2.withColumn('last_mile',lag(col('mile'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule2=rule2.withColumn('last_repair_date',when(col('last_repair_date').isNull(),col('purchase_date')).otherwise(col('last_repair_date')))
    rule2=rule2.withColumn('last_mile',when(col('last_mile').isNull(),lit(0)).otherwise(col('last_mile')))
    rule2=rule2.withColumn('daydiff',datediff(col('repair_date'),col('last_repair_date')))
    rule2=rule2.withColumn('milediff',col('mile')-col('last_mile'))
    rule2=rule2.withColumn('dayofmile',col('milediff')/col('daydiff'))
    rule2=rule2.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    rule3=rule2.withColumn('in_model',lit(0)).dropDuplicates()
    rule3=rule3.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
    rule3=broadcast(rule3)
    
    del tmp
    del rule2

    return rule3

In [40]:
df=spark.sql('select * from clms.skd_final_zhudan')
print(df.select('vin').distinct().count())

165067


In [41]:
model=df.filter(col('mark')=='模型预测')
# print(model.count())
model=model.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
model=broadcast(model)

In [42]:
rule=df.filter(col('mark')=='规则预测')
# print(rule.count())

In [43]:
nobaoyang=df.filter(col('mark')=='非保养数据')
# print(nobaoyang.count())

In [44]:
# print(df.count()==model.count()+rule.count()+nobaoyang.count())

In [45]:
baoyang_vin=model.select('vin').distinct().unionAll(rule.select('vin').distinct())
baoyang_vin=baoyang_vin.withColumn('have_baoyang',lit(1))

In [46]:
if 'have_baoyang' in nobaoyang.columns:
    nobaoyang=nobaoyang.drop('have_baoyang')
nobaoyang=nobaoyang.join(baoyang_vin,on=['vin'],how='left')
nobaoyang=nobaoyang.withColumn('have_baoyang',when(col('have_baoyang').isNull(),lit(0)).otherwise(col('have_baoyang')))
nobaoyang=nobaoyang.filter(col('have_baoyang')==0)
nobaoyang=nobaoyang.drop('have_baoyang')
# print(nobaoyang.count())
# print(nobaoyang.select('vin').distinct().count())

nobaoyang=nobaoyang.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
nobaoyang=broadcast(nobaoyang)

In [47]:
rule2,cleared=clear_rule(rule)
rule2=rule2.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
rule2=broadcast(rule2)

#被天数清洗掉的vin,默认为没有保养数据,将常规保养与首保全都置0
cleared=cleared.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
cleared=broadcast(cleared)

rule3=clear_rule_further(rule2)
rule3=rule3.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
rule3=broadcast(rule3)

del rule2
del rule

In [48]:
same_cols=[x for x in model.columns if x in [y for y in rule3.columns if y in cleared.columns]]
same_cols=[x for x in same_cols if x in nobaoyang.columns]
print(same_cols)

['vin', 'repair_date', 'mile', 'repair_amount', 'repair_amount_pre_discount', 'labor_fee', 'part_fee', 'claim', 'routine_maintenance', 'accessory', 'accident_repair', 'general_repair', 'first_maintance', 'service_act', 'belong_dealer_code', 'city', 'customer_type', 'purchase_date', 'ies_name', 'province', 'family_name', 'city_class', 'member_type', 'whether_bind_wechat', 'car_body_type', 'gearbox', 'crew_protection_system', 'car_class', 'output_year', 'assembly_factory', 'user_tag', 'repair_type', 'mark', 'gender_code', 'member_status', 'member_age']


In [49]:
#重置mark
model=model.withColumn('mark',lit('model'))
rule3=rule3.withColumn('mark',lit('rule'))
cleared=cleared.withColumn('mark',lit('nobaoyang_cleared'))
nobaoyang=nobaoyang.withColumn('mark',lit('nobaoyang'))

In [50]:
dff=model.select(same_cols).unionAll(rule3.select(same_cols)).unionAll(cleared.select(same_cols)).unionAll(nobaoyang.select(same_cols))

In [51]:
dff=dff.withColumn('purchase_date',to_date(col('purchase_date'))).\
withColumn('repair_date',to_date(col('repair_date'))).\
withColumn('repair_day',datediff(col('repair_date'),col('purchase_date')))

dff=dff.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
dff=broadcast(dff)

print(dff.count())
print(dff.select('vin').distinct().count())

543561
165067


In [52]:
veh=spark.sql('select * from clms.skd_vehicle')
print(veh.count())
print(veh.select('vin').distinct().count())

206870
206870


In [53]:
mis1,mis2=get_miss_cols(veh)
print(mis1)

purchase_date


In [54]:
"""重构全量数据"""

'重构全量数据'

In [55]:
veh=veh.withColumnRenamed('purchase_date','purchase_date_in_veh')

In [56]:
same=[x for x in dff.columns if x in veh.columns]
print(same)

['vin', 'belong_dealer_code', 'city', 'customer_type', 'ies_name', 'province', 'family_name', 'city_class', 'member_type', 'whether_bind_wechat', 'car_body_type', 'gearbox', 'crew_protection_system', 'car_class', 'output_year', 'assembly_factory', 'user_tag', 'gender_code', 'member_status', 'member_age']


In [57]:
dff_1=dff.drop('belong_dealer_code', 'city', 'customer_type', 'ies_name', 'province', 'family_name', 'city_class', 'member_type', 'whether_bind_wechat', 'car_body_type', 'gearbox', 'crew_protection_system', 'car_class', 'output_year', 'assembly_factory', 'user_tag', 'gender_code', 'member_status', 'member_age')

In [58]:
all_df=veh.join(dff_1,on=['vin'],how='left')
print(all_df.select('vin').distinct().count())

206870


In [59]:
all_df=all_df.withColumn('purchase_date',when(col('purchase_date').isNull(),col('purchase_date_in_veh')).otherwise(col('purchase_date')))
all_df=all_df.filter(col('purchase_date').isNotNull())
check=all_df.filter(col('purchase_date')>col('repair_date'))
print(check.count())
all_df=all_df.filter(col('purchase_date')<=col('repair_date'))
all_df=all_df.withColumn('repair_date',when(col('repair_date').isNull(),col('purchase_date')).otherwise(col('repair_date')))
all_df=all_df.withColumn('mile',when(col('mile').isNull(),lit(0)).otherwise(col('mile')))
print(all_df.select('vin').distinct().count())

0
165067


In [60]:
# print('开始对人群进行分类')
baoyangdf,nobaoyang=fenlei(all_df,today)
# print(baoyangdf.count())
# print(nobaoyang.count())

baoyangdf=baoyangdf.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
baoyangdf=broadcast(baoyangdf)

nobaoyang=nobaoyang.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
nobaoyang=broadcast(nobaoyang)

查看是否有没有被分配到的人群
165067
165067
[]
查看对应的每一类人群的VIN数量
+--------------------+-----+
|           the_group|  cnt|
+--------------------+-----+
|     有首保,1常规,预测第2次常规|14978|
|    有首保,2次常规,预测第3次常规|12528|
|     有首保,无常规,预测第1次常规|37619|
|    无首保,2次常规,预测第3次常规| 3881|
|     3次及以上常规,预测接下去常规|58240|
|               无保养习惯|17257|
|          无保养记录,预测首保|  615|
|无首保,1次常规,预测第2次常规(购车)|19949|
+--------------------+-----+

None


In [23]:
def get_date3(x,y,z):
    a = x + relativedelta(days = int(y)) + relativedelta(days=int(z))
    return a

def get_date2(x,y):
    a = x + relativedelta(days = int(y))
    return a

get_dates_udf3 = udf(lambda x,y,z:get_date3(x,y,z),DateType())
get_dates_udf2 = udf(lambda x,y:get_date2(x,y),DateType())

In [24]:
def get_next_date_mile(data3,end_id,daydiff_median,milediff_median,dayofmile_median):
    """milediff_pred代表的是公里数差,milediff_%end_id代表的最后一个公里数差,next_milediff代表的是下一次进店间隔的趋势值"""
    
    if 'milediff_pred' not in data3.columns:
        data3=data3.withColumn('milediff_pred',col('milediff_%d'%end_id)*col('next_milediff'))
    if 'dayofmile_pred' not in data3.columns:
        data3=data3.withColumn('dayofmile_pred',col('dayofmile_%d'%end_id)*col('next_dayofmile'))
    
#     print('检查脏数据情况')
#     print('预测出来的公里数差<=0',data3.filter(col('milediff_pred')<=0).select('vin').distinct().count())
    
    data3=data3.withColumn('milediff_pred',when(col('milediff_pred')<=0,lit(milediff_median)).otherwise(col('milediff_pred'))).\
    withColumn('dayofmile_pred',when(col('dayofmile_pred')<=0,lit(dayofmile_median)).otherwise(col('dayofmile_pred')))
    
    data3=data3.withColumn('next_mile',col('mile_%d'%end_id)+col('milediff_pred')).\
    withColumn('next_mile',ceil('next_mile'))
    
    if 'daydiff_pred' in data3.columns:
        data3=data3.drop('daydiff_pred')
        
    data3=data3.withColumn('daydiff_pred',col('milediff_pred')/(col('dayofmile_pred')+0.001)).\
    withColumn('daydiff_pred',ceil('daydiff_pred'))
    
#     print('检查脏数据')
#     print('预测出来的天数差<=0',data3.filter(col('daydiff_pred')<=0).select('vin').distinct().count())
    data3=data3.withColumn('daydiff_pred',when(col('daydiff_pred').isNull(),lit(daydiff_median)).when(col('daydiff_pred')<=0,lit(daydiff_median)).otherwise(col('daydiff_pred')))
    data3=data3.withColumn('next_date',get_dates_udf3(col('purchase_date'),col('repair_day_%d'%end_id),col('daydiff_pred')))
    
    """其中last_baoyang_date代表是最后一次进店保养的时间"""
    data3=data3.withColumn('last_baoyang_date',get_dates_udf2(col('purchase_date'),col('repair_day_%d'%end_id))).\
    withColumn('last_baoyang_mile',col('mile_%d'%end_id))
    
    return data3

#### <font color=red>规则人群0:预测首保

In [25]:
def get_rule_0(nobaoyang):
    rule0=nobaoyang.filter(col('the_group')=='无保养记录,预测首保').\
    withColumn('repair_day_0',lit(0)).\
    withColumn('mile_0',lit(0))
    return rule0

# print('这一个人群比较特殊,购车到首保的天数差,不能直接迭代,因此需要再拟合一个可以往下迭代的数据')
def get_trend_0(baoyangdf):
    """趋势中去掉了脏数据,但是原始自身数据中脏数据还是存在,所以当最终预测出来出现负数的情况下,可能第一考虑的就是原始数据中就有脏数据"""
    df0=baoyangdf.filter(col('first_maintance')==1)
    df0=get_last_run(df0)
    df0=df0.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df00=df0.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df0.groupBy('vin').pivot('baoyang_id',[1]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df00=df00.join(tmp,on='vin',how='left')
    df00=df00.filter((col('daydiff_1')>0)&(col('milediff_1')>0))
    return df00

def nihe_0(df00):
    
    df00=df00.toPandas()
    
    daydiff_median=df00['daydiff_1'].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_1'].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_1'].median()
#     print(dayofmile_median)
    
    key0=['belong_dealer_code']
    nihe0=df00.groupby(key0)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe0.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['first_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField('belong_dealer_code', StringType(), True),
                        StructField("first_daydiff_pred", DoubleType(), True),
                        StructField("first_milediff_pred", DoubleType(), True),
                        StructField("first_dayofmile_pred",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)

    key1=['user_tag']
    nihe1=df00.groupby(key1)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe1.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe1.columns=['second_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField('user_tag', StringType(), True),
                        StructField("second_daydiff_pred", DoubleType(), True),
                        StructField("second_milediff_pred", DoubleType(), True),
                        StructField("second_dayofmile_pred",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    
    key2=['ies_name']
    nihe2=df00.groupby(key2)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe2.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe2.columns=['third_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe2=nihe2.reset_index()
    schema2=StructType([StructField('ies_name', StringType(), True),
                        StructField("third_daydiff_pred", DoubleType(), True),
                        StructField("third_milediff_pred", DoubleType(), True),
                        StructField("third_dayofmile_pred",DoubleType(),True)])
    nihe2=spark.createDataFrame(nihe2,schema=schema2)

    return key0,key1,key2,nihe0,nihe1,nihe2,daydiff_median,milediff_median,dayofmile_median


def get_trend_00(baoyangdf):
    #取出用户第一次常规到首保的天数差/公里数差
    shoubao_vin=baoyangdf.filter(col('first_maintance')==1).select('vin').distinct().withColumn('have_shoubao',lit(1))
    df0=baoyangdf.join(shoubao_vin,on=['vin'],how='left').\
    withColumn('have_shoubao',when(col('have_shoubao').isNull(),lit(0)).otherwise(col('have_shoubao')))
    df0=df0.filter(col('have_shoubao')==1)
    df0=df0.withColumn('day_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df0=df0.filter(col('day_id')<=2).drop('day_id')
    if 'baoyang_id' in df0.columns:
        df0=df0.drop('baoyang_id')
    df0=df0.withColumn('purchase_date',to_date(col('purchase_date'))).\
    withColumn('repair_date',to_date(col('repair_date')))
    df0=df0.withColumn('last_repair_date',lag(col('repair_date'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    df0=df0.withColumn('last_mile',lag(col('mile'),-1).over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    df0=df0.filter(col('last_repair_date').isNotNull())
    df0=df0.withColumn('daydiff',datediff(col('repair_date'),col('last_repair_date'))).\
    withColumn('milediff',col('mile')-col('last_mile')).\
    withColumn('dayofmile',col('milediff')/col('daydiff'))
    df0=df0.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
#     print(df0.count())
#     print(df0.select('vin').distinct().count())
    
    df00=df0.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df0.groupBy('vin').pivot('baoyang_id',[1]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df00=df00.join(tmp,on='vin',how='left')
    df00=df00.filter((col('daydiff_1')>0)&(col('milediff_1')>0))
    
    return df00


def nihe_00(df00):
    
    df00=df00.toPandas()
    
    daydiff_median=df00['daydiff_1'].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_1'].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_1'].median()
#     print(dayofmile_median)
    
    key0=['belong_dealer_code']
    df00['belong_dealer_code']=df00['belong_dealer_code'].astype(str)
    nihe0=df00.groupby(key0)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe0.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['changgui_first_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField('belong_dealer_code', StringType(), True),
                        StructField("changgui_first_daydiff_pred", DoubleType(), True),
                        StructField("changgui_first_milediff_pred", DoubleType(), True),
                        StructField("changgui_first_dayofmile_pred",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)

    key1=['user_tag']
    df00['user_tag']=df00['user_tag'].astype(str)
    nihe1=df00.groupby(key1)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe1.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe1.columns=['changgui_second_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField('user_tag', StringType(), True),
                        StructField("changgui_second_daydiff_pred", DoubleType(), True),
                        StructField("changgui_second_milediff_pred", DoubleType(), True),
                        StructField("changgui_second_dayofmile_pred",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    
    key2=['ies_name']
    df00['ies_name']=df00['ies_name'].astype(str)
    nihe2=df00.groupby(key2)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe2.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe2.columns=['changgui_third_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe2=nihe2.reset_index()
    schema2=StructType([StructField('ies_name', StringType(), True),
                        StructField("changgui_third_daydiff_pred", DoubleType(), True),
                        StructField("changgui_third_milediff_pred", DoubleType(), True),
                        StructField("changgui_third_dayofmile_pred",DoubleType(),True)])
    nihe2=spark.createDataFrame(nihe2,schema=schema2)

    return key0,key1,key2,nihe0,nihe1,nihe2,daydiff_median,milediff_median,dayofmile_median

#### <font color=blue>其中last_baoyang_date指的是最后一次保养数据,然后加上我们预测的接下去的保养天数差后,就得到了最新的next_date

In [26]:
s1=time.time()
rule0=get_rule_0(nobaoyang).dropDuplicates()
trend0=get_trend_0(baoyangdf)
key0,key1,key2,nihe0,nihe1,nihe2,daydiff_median,milediff_median,dayofmile_median=nihe_0(trend0)
rule0=rule0.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left').join(nihe2,on=key2,how='left')

rule0=rule0.withColumn('daydiff_pred',col('first_daydiff_pred')).\
withColumn('daydiff_pred',when((col('daydiff_pred').isNull())&(col('second_daydiff_pred').isNotNull()),col('second_daydiff_pred')).when((col('daydiff_pred').isNull())&(col('third_daydiff_pred').isNotNull()),col('third_daydiff_pred')).otherwise(col('daydiff_pred'))).\
withColumn('milediff_pred',col('first_milediff_pred')).\
withColumn('milediff_pred',when((col('milediff_pred').isNull())&(col('second_milediff_pred').isNotNull()),col('second_milediff_pred')).when((col('milediff_pred').isNull())&(col('third_milediff_pred').isNotNull()),col('third_milediff_pred')).otherwise(col('milediff_pred'))).\
withColumn('dayofmile_pred',col('first_dayofmile_pred')).\
withColumn('dayofmile_pred',when((col('dayofmile_pred').isNull())&(col('second_dayofmile_pred').isNotNull()),col('second_dayofmile_pred')).when((col('dayofmile_pred').isNull())&(col('third_dayofmile_pred').isNotNull()),col('third_dayofmile_pred')).otherwise(col('dayofmile_pred')))

trend00=get_trend_00(baoyangdf)

key0,key1,key2,nihe0,nihe1,nihe2,daydiff_median,milediff_median,dayofmile_median=nihe_00(trend00)
rule0=rule0.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left').join(nihe2,on=key2,how='left')

rule0=rule0.withColumn('changgui_daydiff_pred',col('changgui_first_daydiff_pred')).\
withColumn('changgui_daydiff_pred',when((col('changgui_daydiff_pred').isNull())&(col('changgui_second_daydiff_pred').isNotNull()),col('changgui_second_daydiff_pred')).when((col('changgui_daydiff_pred').isNull())&(col('changgui_third_daydiff_pred').isNotNull()),col('changgui_third_daydiff_pred')).otherwise(col('changgui_daydiff_pred'))).\
withColumn('changgui_milediff_pred',col('changgui_first_milediff_pred')).\
withColumn('changgui_milediff_pred',when((col('changgui_milediff_pred').isNull())&(col('changgui_second_milediff_pred').isNotNull()),col('changgui_second_milediff_pred')).when((col('changgui_milediff_pred').isNull())&(col('changgui_third_milediff_pred').isNotNull()),col('changgui_third_milediff_pred')).otherwise(col('changgui_milediff_pred'))).\
withColumn('changgui_dayofmile_pred',col('changgui_first_dayofmile_pred')).\
withColumn('changgui_dayofmile_pred',when((col('changgui_dayofmile_pred').isNull())&(col('changgui_second_dayofmile_pred').isNotNull()),col('changgui_second_dayofmile_pred')).when((col('changgui_dayofmile_pred').isNull())&(col('changgui_third_dayofmile_pred').isNotNull()),col('changgui_third_dayofmile_pred')).otherwise(col('changgui_dayofmile_pred')))

end_id=0
fin_rule0=get_next_date_mile(rule0,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule0=fin_rule0.withColumn('iterated_daydiff',col('changgui_daydiff_pred')).\
withColumn('iterated_dayofmile',col('changgui_dayofmile_pred')).\
withColumn('iterated_milediff',col('changgui_milediff_pred')).\
withColumn('iterated_start_date',col('next_date')).\
withColumn('iterated_start_mile',col('next_mile'))

need_cols=['vin', 'last_baoyang_date', 'last_baoyang_mile', 'daydiff_pred', 'milediff_pred',
               'dayofmile_pred', 'next_date', 'next_mile', 'purchase_date',
               'iterated_daydiff', 'iterated_start_date','iterated_milediff',
               'iterated_start_mile', 'iterated_dayofmile', 'the_group']
fin_rule0=fin_rule0.select(need_cols).dropDuplicates()

fin_rule0=fin_rule0.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule0=broadcast(fin_rule0)

# print(fin_rule0.count())
# print(fin_rule0.select('vin').distinct().count())

fin_rule0.toPandas().to_csv('rule0.csv',index=False,encoding='utf-8')
check_time(s1)

run time is: 3.3920891284942627


#### <font color=red>规则人群1:没有保养习惯的人

In [27]:
def get_rule_1(nobaoyang1):
    rule1=nobaoyang1.filter(col('the_group')=='无保养习惯').\
    withColumn('repair_day_0',lit(0)).\
    withColumn('mile_0',lit(0))
    return rule1

def get_trend_1(baoyangdf1):
#     print('#取出车龄>3的人群的,在车龄>3之后的第一次进店数据,根据车系喂入')
    dff1=baoyangdf1.filter(col('car_age_bin')>3).withColumn('repair_date',to_date(col('repair_date'))).\
    withColumn('purchase_date',to_date(col('purchase_date'))).\
    withColumn('instore_age',datediff(col('repair_date'),col('purchase_date')))
    
    dff1=dff1.filter(col('instore_age')>540)
    dff1=dff1.withColumn('instore_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    dff1=dff1.filter(col('instore_id')==1).drop('instore_id')
    
    if 'baoyang_id' in dff1.columns:
        dff1=dff1.drop('baoyang_id')
    
    dff1=dff1.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    dff1=get_last_run(dff1)
    
    dff11=dff1.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    for x in ['daydiff','milediff','dayofmile']:
        tmp=dff1.groupBy('vin').pivot('baoyang_id',[1]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        dff11=dff11.join(tmp,on='vin',how='left')
    dff11=dff11.filter((col('daydiff_1')>0)&(col('milediff_1')>0))
    return dff11


def nihe_1(df00):
    
    df00=df00.toPandas()
    
    daydiff_median=df00['daydiff_1'].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_1'].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_1'].median()
#     print(dayofmile_median)
    
    key0=['ies_name']
    nihe0=df00.groupby(key0)['daydiff_1','milediff_1','dayofmile_1'].median()
    nihe0.rename(columns={'daydiff_1':'daydiff_pred','milediff_1':'milediff_pred','dayofmile_1':'dayofmile_pred'},inplace=True)
    nihe0.columns=['first_'+x for x in ['daydiff_pred','milediff_pred','dayofmile_pred']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField(key0[0], StringType(), True),
                        StructField("first_daydiff_pred", DoubleType(), True),
                        StructField("first_milediff_pred", DoubleType(), True),
                        StructField("first_dayofmile_pred",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)

    return key0,nihe0,daydiff_median,milediff_median,dayofmile_median

s1=time.time()
rule1=get_rule_1(nobaoyang).dropDuplicates()
trend1=get_trend_1(baoyangdf)
key0,nihe0,daydiff_median,milediff_median,dayofmile_median=nihe_1(trend1)
rule1=rule1.join(nihe0,on=key0,how='left')

rule1=rule1.withColumn('daydiff_pred',col('first_daydiff_pred')).\
withColumn('milediff_pred',col('first_milediff_pred')).\
withColumn('dayofmile_pred',col('first_dayofmile_pred')).\
withColumn('daydiff_pred',when(col('daydiff_pred').isNull(),lit(daydiff_median)).otherwise(col('daydiff_pred'))).\
withColumn('milediff_pred',when(col('milediff_pred').isNull(),lit(milediff_median)).otherwise(col('milediff_pred'))).\
withColumn('dayofmile_pred',when(col('dayofmile_pred').isNull(),lit(dayofmile_median)).otherwise(col('dayofmile_pred')))

end_id=0
fin_rule1=get_next_date_mile(rule1,end_id,daydiff_median,milediff_median,dayofmile_median)

#由于是无保养习惯人群,所以直接给的540天以后第一次进店后到购车时间的天数差,并以此为迭代数据,否则的话,需要像新车用户那样更加精细化
fin_rule1=fin_rule1.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule1=fin_rule1.select(need_cols).dropDuplicates()

fin_rule1=fin_rule1.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule1=broadcast(fin_rule1)

# print(fin_rule1.count())
# print(fin_rule1.select('vin').distinct().count())

check_time(s1)

fin_rule1.toPandas().to_csv('rule1.csv',index=False,encoding='utf-8')

run time is: 1.3718855500221252


#### <font color=red>规则人群2:有首保,无常规,预测第1次常规

In [28]:
def get_rule_2(baoyangdf):
    rule1=baoyangdf.filter(col('the_group')=='有首保,无常规,预测第1次常规')
    if 'baoyang_id' in rule1.columns:
        rule1=rule1.drop('baoyang_id')
    rule1=rule1.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    rule1=get_last_run(rule1)
    
    rule11=rule1.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule1.groupBy('vin').pivot('baoyang_id',[1]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule11=rule11.join(tmp,on='vin',how='left')
    rule11=rule11.withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500))
    
    rule11=rule11.dropDuplicates()

    return rule11

def get_trend_2(baoyangdf):
    df1=baoyangdf.filter((col('shoubao_times')==1)&(col('changgui_times')>=1))
    
    if 'baoyang_id' in df1.columns:
        df1=df1.drop('baoyang_id')
        
    df1=df1.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df1=df1.filter(col('baoyang_id')<=2)
    df1=get_last_run(df1)
    
    df11=df1.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df1.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df11=df11.join(tmp,on='vin',how='left')
    df11=df11.filter((col('daydiff_1')>0)&(col('daydiff_2')>0)&(col('milediff_1')>0)&(col('milediff_2')>0))
    
    df11=df11.withColumn('daydiff_2_1',col('daydiff_2')/col('daydiff_1')).\
    withColumn('milediff_2_1',col('milediff_2')/(col('milediff_1')+1)).\
    withColumn('dayofmile_2_1',col('dayofmile_2')/(col('dayofmile_1')+0.01)).\
    withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500))

    return df11

def nihe_2(df00):
    df00=df00.toPandas()

    daydiff_trend_median=df00['daydiff_2_1'].median()
#     print(daydiff_trend_median)
    milediff_trend_median=df00['milediff_2_1'].median()
#     print(milediff_trend_median)
    dayofmile_trend_median=df00['dayofmile_2_1'].median()
#     print(dayofmile_trend_median)
    
    daydiff_median=df00['daydiff_%d'%(end_id+1)].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_%d'%(end_id+1)].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_%d'%(end_id+1)].median()
#     print(dayofmile_median)
    
    key0=['daydiff_1_bin','milediff_1_bin']
    nihe0=df00.groupby(key0)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    nihe0.columns=['first_'+x for x in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField(key0[0], StringType(), True),
                        StructField(key0[1], StringType(), True),
                        StructField("first_daydiff_2_1", DoubleType(), True),
                        StructField("first_milediff_2_1", DoubleType(), True),
                        StructField("first_dayofmile_2_1",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)
    
    key1=['daydiff_1_bin']
    nihe1=df00.groupby(key1)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    nihe1.columns=['second_'+x for x in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField(key1[0], StringType(), True),
                        StructField("second_daydiff_2_1", DoubleType(), True),
                        StructField("second_milediff_2_1", DoubleType(), True),
                        StructField("second_dayofmile_2_1",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    return key0,key1,nihe0,nihe1,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median

s1=time.time()
rule2=get_rule_2(baoyangdf)
trend2=get_trend_2(baoyangdf)
key0,key1,nihe0,nihe1,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median=nihe_2(trend2)
rule2=rule2.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left')

rule2=rule2.withColumn('next_daydiff',col('first_daydiff_2_1')).\
withColumn('next_daydiff',when((col('next_daydiff').isNull())&(col('second_daydiff_2_1').isNotNull()),col('second_daydiff_2_1')).\
           when(col('next_daydiff').isNull(),lit(daydiff_trend_median)).otherwise(col('next_daydiff')))
rule2=rule2.withColumn('next_milediff',col('first_milediff_2_1')).\
withColumn('next_milediff',when((col('next_milediff').isNull())&(col('second_milediff_2_1').isNotNull()),col('second_milediff_2_1')).\
           when(col('next_milediff').isNull(),lit(milediff_trend_median)).otherwise(col('next_milediff')))
rule2=rule2.withColumn('next_dayofmile',col('first_dayofmile_2_1')).\
withColumn('next_dayofmile',when((col('next_dayofmile').isNull())&(col('second_dayofmile_2_1').isNotNull()),col('second_dayofmile_2_1')).\
           when(col('next_dayofmile').isNull(),lit(dayofmile_trend_median)).otherwise(col('next_dayofmile')))

end_id=1
fin_rule2=get_next_date_mile(rule2,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule2=fin_rule2.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule2=fin_rule2.select(need_cols).dropDuplicates()
fin_rule2=fin_rule2.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule2=broadcast(fin_rule2)

# print(fin_rule2.count())
# print(fin_rule2.select('vin').distinct().count())
fin_rule2.toPandas().to_csv('rule2.csv',index=False,encoding='utf-8')

check_time(s1)

run time is: 3.943304936091105


#### <font color=red>规则人群3:无首保,1次常规,预测第2次常规(购车)

In [29]:
def get_trend_3(baoyangdf):
    df2=baoyangdf.filter(col('changgui_times')>=2)
    df2=df2.filter(col('first_maintance')!=1)
    
    if 'baoyang_id' in df2.columns:
        df2=df2.drop('baoyang_id')
        
    df2=df2.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df2=df2.filter(col('baoyang_id')<=2)
    df2=get_last_run(df2)
    
    df22=df2.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df2.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df22=df22.join(tmp,on='vin',how='left')
    df22=df22.filter((col('daydiff_1')>0)&(col('daydiff_2')>0)&(col('milediff_1')>0)&(col('milediff_2')>0))

    df22=df22.withColumn('daydiff_2_1',col('daydiff_2')/col('daydiff_1')).\
    withColumn('milediff_2_1',col('milediff_2')/(col('milediff_1')+1)).\
    withColumn('dayofmile_2_1',col('dayofmile_2')/(col('dayofmile_1')+0.01)).\
    withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500))

    return df22

def get_rule_3(baoyangdf):
    rule2=baoyangdf.filter(col('the_group')=='无首保,1次常规,预测第2次常规(购车)')
    rule2=get_last_run(rule2)
    if 'baoyang_id' in rule2.columns:
        rule2=rule2.drop('baoyang_id')
    rule2=rule2.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    rule2=get_last_run(rule2)
    
    rule22=rule2.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule2.groupBy('vin').pivot('baoyang_id',[1]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule22=rule22.join(tmp,on='vin',how='left')
    rule22=rule22.withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500))
    
    rule22=rule22.dropDuplicates()

    return rule22

def nihe_3(df00):
    df00=df00.toPandas()
    
    daydiff_trend_median=df00['daydiff_2_1'].median()
#     print(daydiff_trend_median)
    milediff_trend_median=df00['milediff_2_1'].median()
#     print(milediff_trend_median)
    dayofmile_trend_median=df00['dayofmile_2_1'].median()
#     print(dayofmile_trend_median)
    
    daydiff_median=df00['daydiff_%d'%(end_id+1)].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_%d'%(end_id+1)].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_%d'%(end_id+1)].median()
#     print(dayofmile_median)
    
    key0=['daydiff_1_bin','milediff_1_bin']
    nihe0=df00.groupby(key0)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    nihe0.columns=['first_'+x for x in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField(key0[0], StringType(), True),
                        StructField(key0[1], StringType(), True),
                        StructField("first_daydiff_2_1", DoubleType(), True),
                        StructField("first_milediff_2_1", DoubleType(), True),
                        StructField("first_dayofmile_2_1",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)
    
    key1=['daydiff_1_bin']
    nihe1=df00.groupby(key1)['daydiff_2_1','milediff_2_1','dayofmile_2_1'].median()
    nihe1.columns=['second_'+x for x in ['daydiff_2_1','milediff_2_1','dayofmile_2_1']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField(key1[0], StringType(), True),
                        StructField("second_daydiff_2_1", DoubleType(), True),
                        StructField("second_milediff_2_1", DoubleType(), True),
                        StructField("second_dayofmile_2_1",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    return key0,key1,nihe0,nihe1,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median

s1=time.time()
rule3=get_rule_3(baoyangdf)
trend3=get_trend_3(baoyangdf)
key0,key1,nihe0,nihe1,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median=nihe_3(trend3)
rule3=rule3.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left')

rule3=rule3.withColumn('next_daydiff',col('first_daydiff_2_1')).\
withColumn('next_daydiff',when((col('next_daydiff').isNull())&(col('second_daydiff_2_1').isNotNull()),col('second_daydiff_2_1')).\
           when(col('next_daydiff').isNull(),lit(daydiff_trend_median)).otherwise(col('next_daydiff'))).\
withColumn('next_milediff',col('first_milediff_2_1')).\
withColumn('next_milediff',when((col('next_milediff').isNull())&(col('second_milediff_2_1').isNotNull()),col('second_milediff_2_1')).when((col('next_milediff').isNull())&(col('second_milediff_2_1').isNull()),lit(milediff_trend_median)).otherwise(col('next_milediff'))).\
withColumn('next_dayofmile',col('first_dayofmile_2_1')).\
withColumn('next_dayofmile',when((col('next_dayofmile').isNull())&(col('second_dayofmile_2_1').isNotNull()),col('second_dayofmile_2_1')).when((col('next_dayofmile').isNull())&(col('second_dayofmile_2_1').isNull()),lit(dayofmile_trend_median)).otherwise(col('next_dayofmile')))

end_id=1
fin_rule3=get_next_date_mile(rule3,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule3=fin_rule3.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule3=fin_rule3.select(need_cols).dropDuplicates()

fin_rule3=fin_rule3.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule3=broadcast(fin_rule3)

# print(fin_rule3.count())
# print(fin_rule3.select('vin').distinct().count())

fin_rule3.toPandas().to_csv('rule3.csv',index=False,encoding='utf-8')
check_time(s1)

run time is: 3.976809581120809


#### <font color=red>规则人群4:有首保,1常规,预测第2次常规

In [30]:
def get_trend_4(baoyangdf):
    df2=baoyangdf.filter((col('shoubao_times')==1)&(col('changgui_times')>=2))
    if 'baoyang_id' in df2.columns:
        df2=df2.drop('baoyang_id')
    df2=df2.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df2=df2.filter(col('baoyang_id')<=3)
    df2=get_last_run(df2)
    
    df22=df2.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df2.groupBy('vin').pivot('baoyang_id',[1,2,3]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df22=df22.join(tmp,on='vin',how='left')
    df22=df22.filter((col('daydiff_1')>0)&(col('daydiff_2')>0)&(col('daydiff_3')>0)&(col('milediff_1')>0)&(col('milediff_2')>0)&(col('milediff_3')>0))

    df22=df22.withColumn('daydiff_2_1',col('daydiff_2')/col('daydiff_1')).\
    withColumn('milediff_2_1',col('milediff_2')/(col('milediff_1')+1)).\
    withColumn('dayofmile_2_1',col('dayofmile_2')/(col('dayofmile_1')+0.01)).\
    withColumn('daydiff_3_2',col('daydiff_3')/col('daydiff_2')).\
    withColumn('milediff_3_2',col('milediff_3')/col('milediff_2')).\
    withColumn('dayofmile_3_2',col('dayofmile_3')/col('dayofmile_2')).\
    withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500)).\
    withColumn('daydiff_2_bin',ceil(col('daydiff_2')/15)).\
    withColumn('milediff_2_bin',ceil(col('milediff_2')/2500))
    
    df22=df22.dropDuplicates()
    
    return df22

def get_rule_4(baoyangdf):
    rule3=baoyangdf.filter(col('the_group')=='有首保,1常规,预测第2次常规')
    if 'baoyang_id' in rule3.columns:
        rule3=rule3.drop('baoyang_id')
    rule3=rule3.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    rule3=get_last_run(rule3)
    
    rule33=rule3.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule3.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule33=rule33.join(tmp,on='vin',how='left')
    rule33=rule33.withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500)).\
    withColumn('daydiff_2_bin',ceil(col('daydiff_2')/15)).\
    withColumn('milediff_2_bin',ceil(col('milediff_2')/2500))
    
    rule33=rule33.drop_duplicates()
    
    return rule33

def nihe_4(df00):
    df00=df00.toPandas()
    
    daydiff_trend_median=df00['daydiff_3_2'].median()
#     print(daydiff_trend_median)
    milediff_trend_median=df00['milediff_3_2'].median()
#     print(milediff_trend_median)
    dayofmile_trend_median=df00['dayofmile_3_2'].median()
#     print(dayofmile_trend_median)
    
    daydiff_median=df00['daydiff_%d'%(end_id+1)].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_%d'%(end_id+1)].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_%d'%(end_id+1)].median()
#     print(dayofmile_median)
    
    key0=['daydiff_1_bin','milediff_1_bin','daydiff_2_bin','milediff_2_bin']
    nihe0=df00.groupby(key0)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe0.columns=['first_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField(key0[0], StringType(), True),
                        StructField(key0[1], StringType(), True),
                        StructField(key0[2], StringType(), True),
                        StructField(key0[3], StringType(), True),
                        StructField("first_daydiff_3_2", DoubleType(), True),
                        StructField("first_milediff_3_2", DoubleType(), True),
                        StructField("first_dayofmile_3_2",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)
    
    
    key1=['daydiff_1_bin','daydiff_2_bin','ies_name']
    nihe1=df00.groupby(key1)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe1.columns=['second_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField(key1[0], StringType(), True),
                        StructField(key1[1], StringType(), True),
                        StructField(key1[2], StringType(), True),
                        StructField("second_daydiff_3_2", DoubleType(), True),
                        StructField("second_milediff_3_2", DoubleType(), True),
                        StructField("second_dayofmile_3_2",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    
    key2=['daydiff_1_bin','daydiff_2_bin']
    nihe2=df00.groupby(key2)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe2.columns=['third_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe2=nihe2.reset_index()
    schema2=StructType([StructField(key1[0], StringType(), True),
                        StructField(key1[1], StringType(), True),
                        StructField("third_daydiff_3_2", DoubleType(), True),
                        StructField("third_milediff_3_2", DoubleType(), True),
                        StructField("third_dayofmile_3_2",DoubleType(),True)])
    nihe2=spark.createDataFrame(nihe2,schema=schema2)

    return key0,key1,key2,nihe0,nihe1,nihe2,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median

In [31]:
s1=time.time()
rule4=get_rule_4(baoyangdf)
# print(rule4.select('vin').distinct().count())
trend4=get_trend_4(baoyangdf)
# print(trend4.select('vin').distinct().count())

key0,key1,key2,nihe0,nihe1,nihe2,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median=nihe_4(trend4)
rule4=rule4.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left').join(nihe2,on=key2,how='left')

rule4=rule4.withColumn('next_daydiff',col('first_daydiff_3_2')).\
withColumn('next_daydiff',when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNotNull()),col('second_daydiff_3_2')).\
           when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNull())&(col('third_daydiff_3_2').isNotNull()),col('third_daydiff_3_2')).\
           when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNull())&(col('third_daydiff_3_2').isNull()),lit(daydiff_trend_median)).otherwise(col('next_daydiff'))).\
withColumn('next_milediff',col('first_milediff_3_2')).\
withColumn('next_milediff',when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNotNull()),col('second_milediff_3_2')).\
           when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNull())&(col('third_milediff_3_2').isNotNull()),col('third_milediff_3_2')).\
           when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNull())&(col('third_milediff_3_2').isNull()),lit(milediff_trend_median)).otherwise(col('next_milediff'))).\
withColumn('next_dayofmile',col('first_dayofmile_3_2')).\
withColumn('next_dayofmile',when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNotNull()),col('second_dayofmile_3_2')).\
           when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNull())&(col('third_dayofmile_3_2').isNotNull()),col('third_dayofmile_3_2')).\
           when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNull())&(col('third_dayofmile_3_2').isNull()),lit(dayofmile_trend_median)).otherwise(col('next_dayofmile')))

end_id=2
fin_rule4=get_next_date_mile(rule4,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule4=fin_rule4.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule4=fin_rule4.select(need_cols).dropDuplicates()
fin_rule4=fin_rule4.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule4=broadcast(fin_rule4)

# print(fin_rule4.count())
# print(fin_rule4.select('vin').distinct().count())

fin_rule4.toPandas().to_csv('rule4.csv',index=False,encoding='utf-8')

check_time(s1)

run time is: 4.428108660380046


#### <font color=red>规则人群5:无首保,2次常规,预测第3次常规

In [32]:
def get_trend_5(baoyangdf):
    df2=baoyangdf.filter(col('changgui_times')>=3)
    df2=df2.filter(col('first_maintance')!=1)
    
    if 'baoyang_id' in df2.columns:
        df2=df2.drop('baoyang_id')
    df2=df2.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    df2=df2.filter(col('baoyang_id')<=3)
    df2=get_last_run(df2)
    
    df22=df2.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date').dropDuplicates()
    for x in ['daydiff','milediff','dayofmile']:
        tmp=df2.groupBy('vin').pivot('baoyang_id',[1,2,3]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        df22=df22.join(tmp,on='vin',how='left')
    df22=df22.filter((col('daydiff_1')>0)&(col('daydiff_2')>0)&(col('daydiff_3')>0)&(col('milediff_1')>0)&(col('milediff_2')>0)&(col('milediff_3')>0))

    df22=df22.withColumn('daydiff_2_1',col('daydiff_2')/col('daydiff_1')).\
    withColumn('milediff_2_1',col('milediff_2')/(col('milediff_1')+1)).\
    withColumn('dayofmile_2_1',col('dayofmile_2')/(col('dayofmile_1')+0.01)).\
    withColumn('daydiff_3_2',col('daydiff_3')/col('daydiff_2')).\
    withColumn('milediff_3_2',col('milediff_3')/col('milediff_2')).\
    withColumn('dayofmile_3_2',col('dayofmile_3')/col('dayofmile_2')).\
    withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500)).\
    withColumn('daydiff_2_bin',ceil(col('daydiff_2')/15)).\
    withColumn('milediff_2_bin',ceil(col('milediff_2')/2500))
    
    df22=df22.dropDuplicates()

    return df22

def get_rule_5(baoyangdf):
    rule4=baoyangdf.filter(col('the_group')=='无首保,2次常规,预测第3次常规')
    if 'baoyang_id' in rule4.columns:
        rule4=rule4.drop('baoyang_id')
    rule4=rule4.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(asc('repair_date'))))
    rule4=get_last_run(rule4)
    
    rule44=rule4.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule4.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule44=rule44.join(tmp,on='vin',how='left')
    rule44=rule44.withColumn('daydiff_1_bin',ceil(col('daydiff_1')/15)).\
    withColumn('milediff_1_bin',ceil(col('milediff_1')/2500)).\
    withColumn('daydiff_2_bin',ceil(col('daydiff_2')/15)).\
    withColumn('milediff_2_bin',ceil(col('milediff_2')/2500))
    
    rule44=rule44.drop_duplicates()
    return rule44

def nihe_5(df00):
    df00=df00.toPandas()
    
    daydiff_trend_median=df00['daydiff_3_2'].median()
#     print(daydiff_trend_median)
    milediff_trend_median=df00['milediff_3_2'].median()
#     print(milediff_trend_median)
    dayofmile_trend_median=df00['dayofmile_3_2'].median()
#     print(dayofmile_trend_median)
    
    daydiff_median=df00['daydiff_%d'%(end_id+1)].median()
#     print(daydiff_median)
    milediff_median=df00['milediff_%d'%(end_id+1)].median()
#     print(milediff_median)
    dayofmile_median=df00['dayofmile_%d'%(end_id+1)].median()
#     print(dayofmile_median)
    
    key0=['daydiff_1_bin','milediff_1_bin','daydiff_2_bin','milediff_2_bin']
    nihe0=df00.groupby(key0)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe0.columns=['first_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe0=nihe0.reset_index()
    schema0=StructType([StructField(key0[0], StringType(), True),
                        StructField(key0[1], StringType(), True),
                        StructField(key0[2], StringType(), True),
                        StructField(key0[3], StringType(), True),
                        StructField("first_daydiff_3_2", DoubleType(), True),
                        StructField("first_milediff_3_2", DoubleType(), True),
                        StructField("first_dayofmile_3_2",DoubleType(),True)])
    nihe0=spark.createDataFrame(nihe0,schema=schema0)
    
    
    key1=['daydiff_1_bin','daydiff_2_bin','ies_name']
    nihe1=df00.groupby(key1)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe1.columns=['second_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe1=nihe1.reset_index()
    schema1=StructType([StructField(key1[0], StringType(), True),
                        StructField(key1[1], StringType(), True),
                        StructField(key1[2], StringType(), True),
                        StructField("second_daydiff_3_2", DoubleType(), True),
                        StructField("second_milediff_3_2", DoubleType(), True),
                        StructField("second_dayofmile_3_2",DoubleType(),True)])
    nihe1=spark.createDataFrame(nihe1,schema=schema1)
    
    key2=['daydiff_1_bin','daydiff_2_bin']
    nihe2=df00.groupby(key2)['daydiff_3_2','milediff_3_2','dayofmile_3_2'].median()
    nihe2.columns=['third_'+x for x in ['daydiff_3_2','milediff_3_2','dayofmile_3_2']]
    nihe2=nihe2.reset_index()
    schema2=StructType([StructField(key1[0], StringType(), True),
                        StructField(key1[1], StringType(), True),
                        StructField("third_daydiff_3_2", DoubleType(), True),
                        StructField("third_milediff_3_2", DoubleType(), True),
                        StructField("third_dayofmile_3_2",DoubleType(),True)])
    nihe2=spark.createDataFrame(nihe2,schema=schema2)

    return key0,key1,key2,nihe0,nihe1,nihe2,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median

s1=time.time()
rule5=get_rule_5(baoyangdf)
# print(rule5.select('vin').distinct().count())
trend5=get_trend_5(baoyangdf)
# print(trend5.select('vin').distinct().count())

key0,key1,key2,nihe0,nihe1,nihe2,daydiff_trend_median,milediff_trend_median,dayofmile_trend_median,daydiff_median,milediff_median,dayofmile_median=nihe_5(trend5)
rule5=rule5.join(nihe0,on=key0,how='left').join(nihe1,on=key1,how='left').join(nihe2,on=key2,how='left')

rule5=rule5.withColumn('next_daydiff',col('first_daydiff_3_2')).\
withColumn('next_daydiff',when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNotNull()),col('second_daydiff_3_2')).when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNull())&(col('third_daydiff_3_2').isNotNull()),col('third_daydiff_3_2')).when((col('next_daydiff').isNull())&(col('second_daydiff_3_2').isNull())&(col('third_daydiff_3_2').isNull()),lit(daydiff_trend_median)).otherwise(col('next_daydiff'))).\
withColumn('next_milediff',col('first_milediff_3_2')).\
withColumn('next_milediff',when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNotNull()),col('second_milediff_3_2')).when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNull())&(col('third_milediff_3_2').isNotNull()),col('third_milediff_3_2')).when((col('next_milediff').isNull())&(col('second_milediff_3_2').isNull())&(col('third_milediff_3_2').isNull()),lit(milediff_trend_median)).otherwise(col('next_milediff'))).\
withColumn('next_dayofmile',col('first_dayofmile_3_2')).\
withColumn('next_dayofmile',when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNotNull()),col('second_dayofmile_3_2')).when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNull())&(col('third_dayofmile_3_2').isNotNull()),col('third_dayofmile_3_2')).when((col('next_dayofmile').isNull())&(col('second_dayofmile_3_2').isNull())&(col('third_dayofmile_3_2').isNull()),lit(dayofmile_trend_median)).otherwise(col('next_dayofmile')))

end_id=2
fin_rule5=get_next_date_mile(rule5,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule5=fin_rule5.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule5=fin_rule5.select(need_cols).dropDuplicates()
fin_rule5=fin_rule5.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule5=broadcast(fin_rule5)

# print(fin_rule5.count())
# print(fin_rule5.select('vin').distinct().count())

fin_rule5.toPandas().to_csv('rule5.csv',index=False,encoding='utf-8')

check_time(s1)

run time is: 4.3644527475039165


#### <font color=red>规则人群6:有首保,2次常规,预测第3次常规

In [33]:
def get_rule_6(baoyangdf):
    rule4=baoyangdf.filter(col('the_group')=='有首保,2次常规,预测第3次常规')
    rule4=get_last_run(rule4) 
    
#     print("注意,此处的baoyang_id是倒序!!!")
    if 'baoyang_id' in rule4.columns:
        rule4=rule4.drop('baoyang_id')
    rule4=rule4.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule4=rule4.filter(col('baoyang_id')<=2)
    
    daydiff_median=rule4.approxQuantile('daydiff',[0.5],0.01)[0]
    milediff_median=rule4.approxQuantile('milediff',[0.5],0.01)[0]
    dayofmile_median=rule4.approxQuantile('dayofmile',[0.5],0.01)[0]
#     print(daydiff_median,milediff_median,dayofmile_median)
    
#     print('由于规则中还有天数差的脏数据存在,所以进行平滑一下')
    milediff_right,milediff_left=rule4.approxQuantile('milediff',[0.02,0.98],0.01)
    
    rule4=rule4.withColumn('milediff',when(col('milediff')<milediff_right,lit(milediff_right)).when(col('milediff')>milediff_left,lit(milediff_left)).otherwise(col('milediff')))
    if 'dayofmile' in rule4.columns: #会有负数,所以需要重置
        rule4=rule4.drop('dayofmile')
        rule4=rule4.withColumn('dayofmile',col('milediff')/col('daydiff'))
    
    rule44=rule4.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule4.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule44=rule44.join(tmp,on='vin',how='left')
    
#     print('开始计算vin最近二次日均行驶里程的趋势')
    rule44=rule44.withColumn('milediff_mean',(col('milediff_1')+col('milediff_2'))/2).\
    withColumn('milediff_sub',abs(col('milediff_1')-col('milediff_2'))).\
    withColumn('milediff_per',col('milediff_sub')/col('milediff_mean')).\
    withColumn('daydiff_mean',(col('daydiff_1')+col('daydiff_2'))/2).\
    withColumn('daydiff_sub',abs(col('milediff_1')-col('milediff_2'))).\
    withColumn('daydiff_per',col('daydiff_sub')/col('daydiff_mean')).\
    withColumn('dayofmile_mean',(col('dayofmile_1')+col('dayofmile_2'))/2).\
    withColumn('dayofmile_sub',abs(col('dayofmile_1')-col('dayofmile_2'))).\
    withColumn('dayofmile_per',col('dayofmile_sub')/col('dayofmile_mean'))
    
#     print('注意这里的1和2,与上面的这些批次都是不一样的,是倒序,是倒序,是倒序!')
    rule44=rule44.withColumn('milediff_pred',when(col('milediff_per')<=0.1,col('milediff_1')*0.55+col('milediff_2')*0.45).\
                             when((col('milediff_per')>0.1)&(col('milediff_per')<=0.2),col('milediff_1')*0.6+col('milediff_2')*0.4).\
                             when((col('milediff_per')>0.2)&(col('milediff_per')<=0.3),col('milediff_1')*0.65+col('milediff_2')*0.35).\
                             when((col('milediff_per')>0.3)&(col('milediff_per')<=0.4),col('milediff_1')*0.7+col('milediff_2')*0.3).\
                             when((col('milediff_per')>0.4)&(col('milediff_per')<=0.5),col('milediff_1')*0.75+col('milediff_2')*0.25).\
                             when((col('milediff_per')>0.5)&(col('milediff_per')<=0.6),col('milediff_1')*0.8+col('milediff_2')*0.2).\
                             when((col('milediff_per')>0.6)&(col('milediff_per')<=0.7),col('milediff_1')*0.85+col('milediff_2')*0.15).\
                             when((col('milediff_per')>0.7)&(col('milediff_per')<=0.8),col('milediff_1')*0.9+col('milediff_2')*0.1).\
                             when((col('milediff_per')>0.8)&(col('milediff_per')<=0.9),col('milediff_1')*0.95+col('milediff_2')*0.05).\
                             when((col('milediff_per')>0.9),col('milediff_1')*1))
    
    rule44=rule44.withColumn('daydiff_pred',when(col('daydiff_per')<=0.1,col('daydiff_1')*0.55+col('daydiff_2')*0.45).\
                             when((col('daydiff_per')>0.1)&(col('daydiff_per')<=0.2),col('daydiff_1')*0.6+col('daydiff_2')*0.4).\
                             when((col('daydiff_per')>0.2)&(col('daydiff_per')<=0.3),col('daydiff_1')*0.65+col('daydiff_2')*0.35).\
                             when((col('daydiff_per')>0.3)&(col('daydiff_per')<=0.4),col('daydiff_1')*0.7+col('daydiff_2')*0.3).\
                             when((col('daydiff_per')>0.4)&(col('daydiff_per')<=0.5),col('daydiff_1')*0.75+col('daydiff_2')*0.25).\
                             when((col('daydiff_per')>0.5)&(col('daydiff_per')<=0.6),col('daydiff_1')*0.8+col('daydiff_2')*0.2).\
                             when((col('daydiff_per')>0.6)&(col('daydiff_per')<=0.7),col('daydiff_1')*0.85+col('daydiff_2')*0.15).\
                             when((col('daydiff_per')>0.7)&(col('daydiff_per')<=0.8),col('daydiff_1')*0.9+col('daydiff_2')*0.1).\
                             when((col('daydiff_per')>0.8)&(col('daydiff_per')<=0.9),col('daydiff_1')*0.95+col('daydiff_2')*0.05).\
                             when((col('daydiff_per')>0.9),col('daydiff_1')*1))
    
    rule44=rule44.withColumn('dayofmile_pred',when(col('dayofmile_per')<=0.1,col('dayofmile_1')*0.55+col('dayofmile_2')*0.45).\
                             when((col('dayofmile_per')>0.1)&(col('dayofmile_per')<=0.2),col('dayofmile_1')*0.6+col('dayofmile_2')*0.4).\
                             when((col('dayofmile_per')>0.2)&(col('dayofmile_per')<=0.3),col('dayofmile_1')*0.65+col('dayofmile_2')*0.35).\
                             when((col('dayofmile_per')>0.3)&(col('dayofmile_per')<=0.4),col('dayofmile_1')*0.7+col('dayofmile_2')*0.3).\
                             when((col('dayofmile_per')>0.4)&(col('dayofmile_per')<=0.5),col('dayofmile_1')*0.75+col('dayofmile_2')*0.25).\
                             when((col('dayofmile_per')>0.5)&(col('dayofmile_per')<=0.6),col('dayofmile_1')*0.8+col('dayofmile_2')*0.2).\
                             when((col('dayofmile_per')>0.6)&(col('dayofmile_per')<=0.7),col('dayofmile_1')*0.85+col('dayofmile_2')*0.15).\
                             when((col('dayofmile_per')>0.7)&(col('dayofmile_per')<=0.8),col('dayofmile_1')*0.9+col('dayofmile_2')*0.1).\
                             when((col('dayofmile_per')>0.8)&(col('dayofmile_per')<=0.9),col('dayofmile_1')*0.95+col('dayofmile_2')*0.05).\
                             when((col('dayofmile_per')>0.9),col('dayofmile_1')*1))
    
    rule44=rule44.withColumn('daydiff_pred',ceil('daydiff_pred')).\
    withColumn('milediff_pred',ceil('milediff_pred')).\
    withColumn('dayofmile_pred',ceil('dayofmile_pred'))
    
    return rule44,daydiff_median,milediff_median,dayofmile_median

def get_next_date_mile_67(rule6,end_id,daydiff_median,milediff_median,dayofmile_median):
#     print('检查脏数据情况')
#     print('预测出来的公里数差<=0',rule6.filter(col('milediff_pred')<=0).select('vin').distinct().count())

    rule6=rule6.withColumn('milediff_pred',when(col('milediff_pred')<=0,lit(milediff_median)).otherwise(col('milediff_pred'))).\
    withColumn('dayofmile_pred',when(col('dayofmile_pred')<=0,lit(dayofmile_median)).otherwise(col('dayofmile_pred')))

    rule6=rule6.withColumn('next_mile',col('mile_%d'%end_id)+col('milediff_pred')).\
    withColumn('next_mile',ceil('next_mile'))
    
    if 'daydiff_pred' in rule6.columns:
        rule6=rule6.drop('daydiff_pred')

    rule6=rule6.withColumn('daydiff_pred',col('milediff_pred')/(col('dayofmile_pred')+0.01)).\
    withColumn('daydiff_pred',ceil('daydiff_pred'))

#     print('检查脏数据')
#     print('预测出来的天数差<=0',rule6.filter(col('daydiff_pred')<=0).select('vin').distinct().count())

    rule6=rule6.withColumn('next_date',get_dates_udf3(col('purchase_date'),col('repair_day_1'),col('daydiff_pred')))

    rule6=rule6.withColumn('next_mile',col('mile_1')+col('milediff_pred'))
    rule6=rule6.withColumn('next_mile',ceil('next_mile'))

    rule6=rule6.withColumn('last_baoyang_date',get_dates_udf2(col('purchase_date'),col('repair_day_1')))

    rule6=rule6.withColumn('last_baoyang_mile',col('mile_1'))
    
    return rule6

s1=time.time()
rule6,daydiff_median,milediff_median,dayofmile_median=get_rule_6(baoyangdf)

end_id=1
fin_rule6=get_next_date_mile_67(rule6,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule6=fin_rule6.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule6=fin_rule6.select(need_cols).dropDuplicates()
fin_rule6=fin_rule6.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule6=broadcast(fin_rule6)

# print(fin_rule6.count())
# print(fin_rule6.select('vin').distinct().count())

fin_rule6.toPandas().to_csv('rule6.csv',index=False,encoding='utf-8')

check_time(s1)

run time is: 3.5460384686787925


#### <font color=red>规则人群7:3次及以上常规,预测接下去常规

In [34]:
def get_rule_7(baoyangdf):
    rule4=baoyangdf.filter(col('the_group')=='3次及以上常规,预测接下去常规')
    rule4=get_last_run(rule4)
    
    if 'baoyang_id' in rule4.columns:
        rule4=rule4.drop('baoyang_id')
    
#     print("注意,此处的baoyang_id是倒序!!!")
    rule4=rule4.withColumn('baoyang_id',row_number().over(Window.partitionBy('vin').orderBy(desc('repair_date'))))
    rule4=rule4.filter(col('baoyang_id')<=2)
    
    daydiff_median=rule4.approxQuantile('daydiff',[0.5],0.01)[0]
    milediff_median=rule4.approxQuantile('milediff',[0.5],0.01)[0]
    dayofmile_median=rule4.approxQuantile('dayofmile',[0.5],0.01)[0]
#     print(daydiff_median,milediff_median,dayofmile_median)
    
#     print('由于规则中还有天数差的脏数据存在,所以进行平滑一下')
    milediff_right,milediff_left=rule4.approxQuantile('milediff',[0.02,0.98],0.01)
    
    rule4=rule4.withColumn('milediff',when(col('milediff')<milediff_right,lit(milediff_right)).when(col('milediff')>milediff_left,lit(milediff_left)).otherwise(col('milediff')))
    if 'dayofmile' in rule4.columns: #会有负数,所以需要重置
        rule4=rule4.withColumn('dayofmile',col('milediff')/col('daydiff'))
    
    rule44=rule4.select('vin','belong_dealer_code','ies_name','user_tag','purchase_date','the_group').dropDuplicates()
    
    for x in ['daydiff','milediff','dayofmile','repair_day','mile']:
        tmp=rule4.groupBy('vin').pivot('baoyang_id',[1,2]).agg(mean(x))
        cols=tmp.columns
        cols.remove('vin')
        for y in cols:
            tmp=tmp.withColumnRenamed(y,'%s_%s'%(x,y))
        rule44=rule44.join(tmp,on='vin',how='left')
    
#     print('开始计算vin最近二次日均行驶里程的趋势')
    rule44=rule44.withColumn('milediff_mean',(col('milediff_1')+col('milediff_2'))/2).\
    withColumn('milediff_sub',abs(col('milediff_1')-col('milediff_2'))).\
    withColumn('milediff_per',col('milediff_sub')/col('milediff_mean')).\
    withColumn('daydiff_mean',(col('daydiff_1')+col('daydiff_2'))/2).\
    withColumn('daydiff_sub',abs(col('milediff_1')-col('milediff_2'))).\
    withColumn('daydiff_per',col('daydiff_sub')/col('daydiff_mean')).\
    withColumn('dayofmile_mean',(col('dayofmile_1')+col('dayofmile_2'))/2).\
    withColumn('dayofmile_sub',abs(col('dayofmile_1')-col('dayofmile_2'))).\
    withColumn('dayofmile_per',col('dayofmile_sub')/col('dayofmile_mean'))
    
#     print('注意这里的1和2,与上面的这些批次都是不一样的,是倒序,是倒序,是倒序!')
    rule44=rule44.withColumn('milediff_pred',when(col('milediff_per')<=0.1,col('milediff_1')*0.55+col('milediff_2')*0.45).\
                             when((col('milediff_per')>0.1)&(col('milediff_per')<=0.2),col('milediff_1')*0.6+col('milediff_2')*0.4).\
                             when((col('milediff_per')>0.2)&(col('milediff_per')<=0.3),col('milediff_1')*0.65+col('milediff_2')*0.35).\
                             when((col('milediff_per')>0.3)&(col('milediff_per')<=0.4),col('milediff_1')*0.7+col('milediff_2')*0.3).\
                             when((col('milediff_per')>0.4)&(col('milediff_per')<=0.5),col('milediff_1')*0.75+col('milediff_2')*0.25).\
                             when((col('milediff_per')>0.5)&(col('milediff_per')<=0.6),col('milediff_1')*0.8+col('milediff_2')*0.2).\
                             when((col('milediff_per')>0.6)&(col('milediff_per')<=0.7),col('milediff_1')*0.85+col('milediff_2')*0.15).\
                             when((col('milediff_per')>0.7)&(col('milediff_per')<=0.8),col('milediff_1')*0.9+col('milediff_2')*0.1).\
                             when((col('milediff_per')>0.8)&(col('milediff_per')<=0.9),col('milediff_1')*0.95+col('milediff_2')*0.05).\
                             when((col('milediff_per')>0.9),col('milediff_1')*1))
    
    rule44=rule44.withColumn('daydiff_pred',when(col('daydiff_per')<=0.1,col('daydiff_1')*0.55+col('daydiff_2')*0.45).\
                             when((col('daydiff_per')>0.1)&(col('daydiff_per')<=0.2),col('daydiff_1')*0.6+col('daydiff_2')*0.4).\
                             when((col('daydiff_per')>0.2)&(col('daydiff_per')<=0.3),col('daydiff_1')*0.65+col('daydiff_2')*0.35).\
                             when((col('daydiff_per')>0.3)&(col('daydiff_per')<=0.4),col('daydiff_1')*0.7+col('daydiff_2')*0.3).\
                             when((col('daydiff_per')>0.4)&(col('daydiff_per')<=0.5),col('daydiff_1')*0.75+col('daydiff_2')*0.25).\
                             when((col('daydiff_per')>0.5)&(col('daydiff_per')<=0.6),col('daydiff_1')*0.8+col('daydiff_2')*0.2).\
                             when((col('daydiff_per')>0.6)&(col('daydiff_per')<=0.7),col('daydiff_1')*0.85+col('daydiff_2')*0.15).\
                             when((col('daydiff_per')>0.7)&(col('daydiff_per')<=0.8),col('daydiff_1')*0.9+col('daydiff_2')*0.1).\
                             when((col('daydiff_per')>0.8)&(col('daydiff_per')<=0.9),col('daydiff_1')*0.95+col('daydiff_2')*0.05).\
                             when((col('daydiff_per')>0.9),col('daydiff_1')*1))
    
    rule44=rule44.withColumn('dayofmile_pred',when(col('dayofmile_per')<=0.1,col('dayofmile_1')*0.55+col('dayofmile_2')*0.45).\
                             when((col('dayofmile_per')>0.1)&(col('dayofmile_per')<=0.2),col('dayofmile_1')*0.6+col('dayofmile_2')*0.4).\
                             when((col('dayofmile_per')>0.2)&(col('dayofmile_per')<=0.3),col('dayofmile_1')*0.65+col('dayofmile_2')*0.35).\
                             when((col('dayofmile_per')>0.3)&(col('dayofmile_per')<=0.4),col('dayofmile_1')*0.7+col('dayofmile_2')*0.3).\
                             when((col('dayofmile_per')>0.4)&(col('dayofmile_per')<=0.5),col('dayofmile_1')*0.75+col('dayofmile_2')*0.25).\
                             when((col('dayofmile_per')>0.5)&(col('dayofmile_per')<=0.6),col('dayofmile_1')*0.8+col('dayofmile_2')*0.2).\
                             when((col('dayofmile_per')>0.6)&(col('dayofmile_per')<=0.7),col('dayofmile_1')*0.85+col('dayofmile_2')*0.15).\
                             when((col('dayofmile_per')>0.7)&(col('dayofmile_per')<=0.8),col('dayofmile_1')*0.9+col('dayofmile_2')*0.1).\
                             when((col('dayofmile_per')>0.8)&(col('dayofmile_per')<=0.9),col('dayofmile_1')*0.95+col('dayofmile_2')*0.05).\
                             when((col('dayofmile_per')>0.9),col('dayofmile_1')*1))
    
    rule44=rule44.withColumn('daydiff_pred',ceil('daydiff_pred')).\
    withColumn('milediff_pred',ceil('milediff_pred'))
    
    return rule44,daydiff_median,milediff_median,dayofmile_median

In [35]:
s1=time.time()
rule7,daydiff_median,milediff_median,dayofmile_median=get_rule_7(baoyangdf)

end_id=1
fin_rule7=get_next_date_mile_67(rule7,end_id,daydiff_median,milediff_median,dayofmile_median)

fin_rule7=fin_rule7.withColumn('iterated_daydiff',col('daydiff_pred')).\
withColumn('iterated_dayofmile',col('dayofmile_pred')).\
withColumn('iterated_milediff',col('milediff_pred')).\
withColumn('iterated_start_date',col('last_baoyang_date')).\
withColumn('iterated_start_mile',col('last_baoyang_mile'))

fin_rule7=fin_rule7.select(need_cols).dropDuplicates()
fin_rule7=fin_rule7.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
fin_rule7=broadcast(fin_rule7)

# print(fin_rule7.count())
# print(fin_rule7.select('vin').distinct().count())

fin_rule7.toPandas().to_csv('rule7.csv',index=False,encoding='utf-8')

check_time(s1)

run time is: 3.5723326802253723


In [36]:
print((time.time()-ft)/60)

31.09823716878891


### <font color=red>合并规则

In [ ]:
rule=fin_rule0.unionAll(fin_rule1).unionAll(fin_rule2).unionAll(fin_rule3).unionAll(fin_rule4).unionAll(fin_rule5).unionAll(fin_rule6).unionAll(fin_rule7)
rule=rule.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
rule=broadcast(rule)
# print(rule.count())
# print(rule.select('vin').distinct().count())

In [38]:
print('得到模型人群')
# model=spark.sql('select * from clms.next_model_result')
model_1=spark.sql('select * from clms.fin_result_2_5')
model_2=spark.sql('select * from clms.fin_result_9')
model=model_1.unionAll(model_2)
model=model.filter(col('fin_mileofday')>0)
model=model.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
model=broadcast(model)

得到模型人群


In [39]:
model_vin=model.select('vin').distinct().withColumn('is_in_model',lit(1))

rule=rule.join(model_vin,on='vin',how='left')
rule=rule.withColumn('is_in_model',when(col('is_in_model').isNull(),lit(0)).otherwise(col('is_in_model')))

the_rule=rule.filter(col('is_in_model')==0)
the_rule=the_rule.drop('is_in_model')
the_rule=the_rule.withColumn('next_baoyang_date_fin',col('next_date')).\
withColumn('next_baoyang_mile_fin',col('next_mile')).\
withColumn('method',lit('rule'))

#模型
model=model.withColumnRenamed('last_repair_date','last_baoyang_date').\
withColumnRenamed('last_repair_mile','last_baoyang_mile').\
withColumn('next_baoyang_date_fin',col('next_repair_date')).\
withColumn('next_baoyang_mile_fin',col('next_repair_mile')).\
withColumn('iterated_daydiff',col('daydiff_diff')).\
withColumn('iterated_dayofmile',col('fin_mileofday')).\
withColumn('iterated_milediff',col('milediff_diff')).\
withColumn('method',lit('model'))

same_cols=[x for x in the_rule.columns if x in model.columns]
print(same_cols)

#合并
all_df=the_rule.select(same_cols).unionAll(model.select(same_cols))

if 'iterated_daydiff' in all_df.columns:
    all_df=all_df.drop('iterated_daydiff')

all_df=all_df.withColumn('iterated_daydiff',col('iterated_milediff')/col('iterated_dayofmile'))

all_df=all_df.persist(pyspark.StorageLevel.MEMORY_AND_DISK_SER)
all_df=broadcast(all_df)

['vin', 'last_baoyang_date', 'last_baoyang_mile', 'purchase_date', 'iterated_daydiff', 'iterated_milediff', 'iterated_dayofmile', 'next_baoyang_date_fin', 'next_baoyang_mile_fin', 'method']


### <font color=red>开始迭代

In [40]:
def get_two_dates(df):
    if 'today' in df.columns:
        df=df.drop('today')
    df=df.withColumn('today',to_date(lit(today)))
    
    #拆分成二个部分处理
    df1=df.filter(col('next_baoyang_date_fin')>col('today')).\
    withColumn('next_baoyang_date_fin_after_today',col('next_baoyang_date_fin')).\
    withColumn('next_baoyang_mile_fin_after_today',col('next_baoyang_mile_fin')).\
    withColumn('iterated_id',lit(0)).\
    withColumn('rounds',lit(0))
    
    df2=df.filter(col('next_baoyang_date_fin')<=col('today')).\
    withColumn('rounds',ceil(datediff(col('today'),col('next_baoyang_date_fin'))/col('iterated_daydiff'))).\
    withColumn('next_baoyang_date_fin_after_today',get_dates_udf3(col('next_baoyang_date_fin'),col('iterated_daydiff'),col('rounds'))).\
    withColumn('next_baoyang_mile_fin_after_today',col('next_baoyang_mile_fin')+col('iterated_milediff')*col('rounds')).\
    withColumn('next_baoyang_mile_fin_after_today',ceil('next_baoyang_mile_fin_after_today')).withColumn('iterated_id',col('rounds')).\
    withColumn('rounds',col('rounds').cast('Double')).\
    withColumn('next_baoyang_mile_fin_after_today',col('next_baoyang_mile_fin_after_today').cast('Double')).\
    withColumn('iterated_id',col('iterated_id').cast('Double'))
    
    same_cols=[x for x in df1.columns if x in df2.columns]
    df=df1.select(same_cols).unionAll(df2.select(same_cols))
    
    tongyi_cols=['vin', 'purchase_date', 'last_baoyang_date', 'last_baoyang_mile',
                 'iterated_daydiff', 'iterated_dayofmile','iterated_milediff',
                 'next_baoyang_date_fin', 'next_baoyang_mile_fin',
                 'next_baoyang_date_fin_after_today','next_baoyang_mile_fin_after_today','iterated_id','method']
    df=df.select(tongyi_cols).dropDuplicates()
    return df

In [41]:
pred=get_two_dates(all_df)

In [42]:
pred.registerTempTable('pred_reg')

spark.sql('drop table if exists clms.next_final_result')

spark.sql('create table if not exists clms.next_final_result (vin string,purchase_date date,\
last_baoyang_date date,last_baoyang_mile double,iterated_daydiff double,iterated_dayofmile double,\
iterated_milediff double,next_baoyang_date_fin date,next_baoyang_mile_fin double,\
next_baoyang_date_fin_after_today date,next_baoyang_mile_fin_after_today double,iterated_id string,method string)')

spark.sql('insert overwrite table clms.next_final_result select * from pred_reg')

DataFrame[]

In [ ]:
print((time.time()-ft)/60)